# Tarea DataFrames PySpark 

**PROCESAMIENTO DE GRANDES BASES DE DATOS**

PÉREZ ROSAS LUIS ALFREDO | 325057368 | 2025-B | GRUPO: 2 

##### Importación de librerias requeridas

In [1]:
import findspark
findspark.init()

import pandas as pd
import pyspark

##### Creación de sesion de Spark

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local[*]")\
        .appName('PySpark_Df')\
        .getOrCreate()

##### Lectura de archivo .csv, volcado de datos en dataframe y muestra rapida de datos

In [4]:
fifa_df = spark.read.csv ("WorldCupPlayers.csv",
                          inferSchema = True,
                          header = True)
fifa_df.show ()

+-------+-------+-------------+-------------------+-------+------------+-----------------+--------+---------+
|RoundID|MatchID|Team Initials|         Coach Name|Line-up|Shirt Number|      Player Name|Position|    Event|
+-------+-------+-------------+-------------------+-------+------------+-----------------+--------+---------+
|    201|   1096|          FRA|CAUDRON Raoul (FRA)|      S|           0|   EPIFANIO SAGUN|      GK|     NULL|
|    201|   1096|          MEX|   LUQUE Juan (MEX)|      S|           0|   ERIBERTO ROSAS|      GK|     NULL|
|    201|   1096|          FRA|CAUDRON Raoul (FRA)|      S|           0|    MARIANA GONZE|    NULL|     G40'|
|    201|   1096|          MEX|   LUQUE Juan (MEX)|      S|           0|     JUANA ANDANA|    NULL|     G70'|
|    201|   1096|          FRA|CAUDRON Raoul (FRA)|      S|           0| ERIBERT WILLIAMS|    NULL|     NULL|
|    201|   1096|          MEX|   LUQUE Juan (MEX)|      S|           0|     Rafael GARZA|       C|     NULL|
|    201| 

##### Muestra de estructura / esquema del dataframe creado

In [5]:
fifa_df.printSchema()

root
 |-- RoundID: integer (nullable = true)
 |-- MatchID: integer (nullable = true)
 |-- Team Initials: string (nullable = true)
 |-- Coach Name: string (nullable = true)
 |-- Line-up: string (nullable = true)
 |-- Shirt Number: integer (nullable = true)
 |-- Player Name: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- Event: string (nullable = true)



##### Conteo de los registros existentes en el dataframe

In [6]:
fifa_df.count()

37784

#### Estadisticos base de la columna Player Name; al ser una variable cadena no es posible obtener algunos estadisticos

In [9]:
fifa_df.describe('Player Name').show()

+-------+-----------+
|summary|Player Name|
+-------+-----------+
|  count|      37784|
|   mean|       NULL|
| stddev|       NULL|
|    min|    ?URI?I?|
|    max|       �ZIL|
+-------+-----------+



#### Relacion agrupada de Jugadores y Numero de playera

In [10]:
fifa_df.select('Player Name','Shirt Number').distinct().show()

+-------------------+------------+
|        Player Name|Shirt Number|
+-------------------+------------+
|     Marcel CAPELLE|           0|
|     Demetrio NEYRA|           0|
|       Rudolf RAFTL|           0|
|   Sigmund HARINGER|           0|
|           BALTAZAR|           0|
|       Oscar MIGUEZ|           0|
|       Willy KERNEN|          14|
|        Kurt HAMRIN|           7|
|    Rudolf SZANWALD|           1|
|       Kurt SCHMIED|          12|
|      Bobby TRAINOR|          22|
|      Domingo PEREZ|           7|
|     Honorino LANDA|           9|
|     Jimmy ARMFIELD|           2|
|    Anibal TARABINI|          22|
|     Andrej KVASNAK|           6|
|     Staffan TAPPER|          14|
|      Enrique WOLFF|          20|
|Ramon Maria CALDERE|          18|
|             MORATO|          13|
+-------------------+------------+
only showing top 20 rows



#### Filtro de datos de dataframe, cuenta el numero de registros donde RoundID es igual a 201

In [17]:
fifa_df.filter(fifa_df.RoundID =='201').count()

566

#### Filtro de datos donde posicion es GK y MatchID es 1096

In [28]:
fifa_df.filter((fifa_df.Position == 'GK') & (fifa_df.MatchID=="1096")).show()

+-------+-------+-------------+-------------------+-------+------------+--------------+--------+-----+
|RoundID|MatchID|Team Initials|         Coach Name|Line-up|Shirt Number|   Player Name|Position|Event|
+-------+-------+-------------+-------------------+-------+------------+--------------+--------+-----+
|    201|   1096|          FRA|CAUDRON Raoul (FRA)|      S|           0|EPIFANIO SAGUN|      GK| NULL|
|    201|   1096|          MEX|   LUQUE Juan (MEX)|      S|           0|ERIBERTO ROSAS|      GK| NULL|
+-------+-------+-------------+-------------------+-------+------------+--------------+--------+-----+



#### Creación de una tabla / vista temporal tomando como fuente de datos el dataframe fifa_df y posterior consulta de tipo SQL donde MatchID es mayor o igual 2000

In [34]:
fifa_df.createOrReplaceTempView("temp_table")

spark.sql("select * from temp_table where MatchID >= 2000").show()

+-------+-------+-------------+-------------------+-------+------------+--------------------+--------+---------+
|RoundID|MatchID|Team Initials|         Coach Name|Line-up|Shirt Number|         Player Name|Position|    Event|
+-------+-------+-------------+-------------------+-------+------------+--------------------+--------+---------+
|    262|   2003|          FRG|SCHOEN Helmut (FRG)|      S|           1|          Sepp MAIER|      GK|     NULL|
|    262|   2003|          CHI|  ALAMOS Luis (CHI)|      S|           1|   Leopoldo VALLEJOS|      GK|     NULL|
|    262|   2003|          FRG|SCHOEN Helmut (FRG)|      S|           2|         Berti VOGTS|    NULL|     NULL|
|    262|   2003|          CHI|  ALAMOS Luis (CHI)|      S|           2|      Rolando GARCIA|    NULL|      Y1'|
|    262|   2003|          FRG|SCHOEN Helmut (FRG)|      S|           3|       Paul BREITNER|    NULL|     G18'|
|    262|   2003|          CHI|  ALAMOS Luis (CHI)|      S|           3|    Alberto QUINTANO|   